# Landing Feedly Opportunities

Pipeline de chargement des opportunites Feedly enrichies vers le Lakehouse Fabric.

## Architecture Complete

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                           AZURE FUNCTIONS PIPELINE                          │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  Feedly API ──► APIM ──► /api/feed ──► q-raw-events                        │
│                                              │                              │
│                                              ▼                              │
│                                       enrich_event                          │
│                                    (lac-weak-signals)                       │
│                                              │                              │
│                              ┌───────────────┴───────────────┐              │
│                              ▼                               ▼              │
│                     AI Search (upsert)              q-enriched-events       │
│                              │                               │              │
│                              │                               ▼              │
│                              │                        analyze_event         │
│                              │                     (lac-analyst-leads)      │
│                              │                               │              │
│                              ◄───────────────────────────────┘              │
│                              │        AI Search (merge)                     │
│                              │                               │              │
│                              │                               ▼              │
│                              │                      q-opportunities         │
│                              │                      (si auditOpportunity)   │
└──────────────────────────────┼──────────────────────────────────────────────┘
                               │
                               ▼
┌─────────────────────────────────────────────────────────────────────────────┐
│                           FABRIC LAKEHOUSE                                  │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  Ce Notebook ──► AI Search ──► landing_feedly_opportunities (Delta)        │
│                                              │                              │
│                                              ▼                              │
│                                      Power BI Dashboard                     │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

## Modes de chargement

| Mode | Table cible | Action | Quand l'utiliser |
|------|-------------|--------|------------------|
| `one_time` | `landing_feedly_opportunities` | OVERWRITE | Deploiement initial |
| `incremental` | `landing_feedly_opportunities` | MERGE (upsert) | CRON hebdo |
| `full_refresh` | `curated_feedly_opportunities` | OVERWRITE | Ad hoc / mensuel |

## 1. Configuration

In [ ]:
# =============================================================================
# CONFIGURATION
# =============================================================================

# Mode de chargement: "one_time", "incremental", "full_refresh"
LOAD_MODE = "one_time"  # Premiere execution: charger tout l'historique

# Azure AI Search
AI_SEARCH_ENDPOINT = "https://lac-feedly-search.search.windows.net"
AI_SEARCH_INDEX = "raw_events"
AI_SEARCH_KEY = "<YOUR_AI_SEARCH_KEY>"  # TODO: Utiliser Key Vault en prod

# Lakehouse Tables
TABLE_LANDING = "landing_feedly_opportunities"      # Table principale (incremental)
TABLE_CURATED = "curated_feedly_opportunities"      # Table consolidee (full refresh)

# Incremental settings
INCREMENTAL_HOURS = 24 * 7  # 1 semaine (168 heures)

print(f"Mode: {LOAD_MODE}")
print(f"Table landing: {TABLE_LANDING}")
print(f"Table curated: {TABLE_CURATED}")
print(f"Incremental window: {INCREMENTAL_HOURS} heures ({INCREMENTAL_HOURS//24} jours)")

## 2. Imports et Setup

In [ ]:
import requests
import json
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    StructType, StructField, StringType, IntegerType, 
    DoubleType, BooleanType, TimestampType, LongType
)
from pyspark.sql.functions import (
    col, from_unixtime, to_timestamp, current_timestamp,
    lit, when, coalesce
)
from datetime import datetime, timedelta
from delta.tables import DeltaTable

spark = SparkSession.builder.getOrCreate()
print("✅ Spark session ready")

## 3. Schéma de la table

In [ ]:
# Schema unifie pour landing et curated
# Synchronise avec le schema AI Search (recreate_index.py)
# + colonnes ingestion_week et deduplication
schema = StructType([
    # Identifiant unique
    StructField("id", StringType(), False),
    
    # Article metadata
    StructField("article_url", StringType(), True),
    StructField("article_title", StringType(), True),
    StructField("article_content", StringType(), True),
    StructField("publication_date", TimestampType(), True),
    StructField("language", StringType(), True),
    StructField("origin", StringType(), True),
    StructField("entities", StringType(), True),
    StructField("topics", StringType(), True),
    
    # Vertical (market segment)
    StructField("vertical", StringType(), True),
    
    # Venue information
    StructField("venue_name", StringType(), True),
    StructField("city", StringType(), True),
    StructField("country", StringType(), True),
    StructField("zone", StringType(), True),  # EMEA, APAC, AMERICAS
    StructField("venue_type", StringType(), True),
    StructField("capacity", IntegerType(), True),
    
    # Project information
    StructField("project_type", StringType(), True),
    StructField("project_phase", StringType(), True),
    StructField("opening_year", IntegerType(), True),
    StructField("opening_date", TimestampType(), True),
    
    # Financial information
    StructField("investment", DoubleType(), True),
    StructField("investment_currency", StringType(), True),
    
    # Stakeholders
    StructField("investor_owner_management", StringType(), True),
    StructField("architect_consultant_contractor", StringType(), True),
    
    # Competitor intelligence
    StructField("competitor_name_main", StringType(), True),
    StructField("competitor_name_other", StringType(), True),
    StructField("key_products_installed", StringType(), True),
    StructField("system_integrator", StringType(), True),
    StructField("other_key_players", StringType(), True),
    
    # Additional info
    StructField("additional_information", StringType(), True),
    
    # Analysis results (Agent lac-analyst-leads)
    StructField("evaluation_score", IntegerType(), True),  # Score 0-100
    StructField("audit_opportunity", BooleanType(), True),
    StructField("audit_opportunity_reason", StringType(), True),
    
    # Metadata
    StructField("crawled_at", TimestampType(), True),  # Quand Feedly a crawle
    StructField("loaded_at", TimestampType(), True),   # Quand charge dans Fabric
    StructField("updated_at", TimestampType(), True),  # Derniere mise a jour
    StructField("ingestion_week", StringType(), True), # Semaine d'ingestion (ex: "2024-W49")
    
    # Deduplication (rempli par deduplicate_weekly.ipynb)
    StructField("is_duplicate", BooleanType(), True),           # True si doublon confirme (score >= 0.90)
    StructField("is_suspected_duplicate", BooleanType(), True), # True si zone grise (0.85 <= score < 0.90)
    StructField("duplicate_of", StringType(), True),            # ID de l'article original
    StructField("duplicate_score", DoubleType(), True)          # Score de similarite (0.0 a 1.0)
])

print(f"Schema defini avec {len(schema.fields)} colonnes")
print("  - Nouvelles colonnes: ingestion_week, is_duplicate, is_suspected_duplicate, duplicate_of, duplicate_score")

## 4. Fonctions d'extraction AI Search

In [ ]:
def fetch_documents_from_ai_search(
    endpoint: str, 
    index: str, 
    api_key: str, 
    mode: str = "incremental",
    hours_back: int = 168,
    batch_size: int = 1000
):
    """
    Recupere les documents depuis Azure AI Search.
    
    Args:
        mode: "one_time" | "incremental" | "full_refresh"
        hours_back: Pour incremental, nombre d'heures a regarder en arriere (defaut: 168 = 1 semaine)
    
    Note: Le filtre sur crawled utilise Int64 (millisecondes).
    """
    url = f"{endpoint}/indexes/{index}/docs/search?api-version=2023-11-01"
    headers = {
        "Content-Type": "application/json",
        "api-key": api_key
    }
    
    # Construire le filtre selon le mode
    filter_clause = None
    if mode == "incremental":
        # Filtre sur crawled (timestamp en millisecondes - Int64)
        cutoff_ms = int((datetime.now() - timedelta(hours=hours_back)).timestamp() * 1000)
        filter_clause = f"crawled ge {cutoff_ms}"
        print(f"Mode incremental: crawled >= {datetime.fromtimestamp(cutoff_ms/1000)} ({hours_back}h)")
    elif mode == "one_time":
        print("Mode one-time: chargement de TOUT l'historique")
    elif mode == "full_refresh":
        print("Mode full refresh: rechargement complet")
    
    all_documents = []
    skip = 0
    
    # Tous les champs du schema AI Search
    select_fields = [
        # Technical
        "id", "url", "origin", "published", "crawled", "language", "sourceId",
        # Content
        "title", "content", "entities", "topics",
        # Business Intelligence
        "publicationDate", "vertical",
        # Venue
        "venueName", "city", "country", "zone", "venueType", "capacity",
        # Project
        "projectType", "projectPhase", "openingYear", "openingDate",
        # Financial
        "investment", "investmentCurrency",
        # Stakeholders
        "investorOwnerManagement", "architectConsultantContractor",
        # Competitor intelligence
        "competitorNameMain", "competitorNameOther", "keyProductsInstalled",
        "systemIntegrator", "otherKeyPlayers",
        # Additional
        "additionalInformation",
        # Analysis
        "evaluationScore", "auditOpportunity", "auditOpportunityReason"
    ]
    
    while True:
        body = {
            "search": "*",
            "top": batch_size,
            "skip": skip,
            "select": ",".join(select_fields),
            "orderby": "crawled desc"
        }
        
        if filter_clause:
            body["filter"] = filter_clause
        
        response = requests.post(url, headers=headers, json=body)
        response.raise_for_status()
        
        result = response.json()
        documents = result.get("value", [])
        
        if not documents:
            break
            
        all_documents.extend(documents)
        skip += batch_size
        
        print(f"   Fetched {len(all_documents)} documents...")
        
        if len(documents) < batch_size:
            break
    
    print(f"Total: {len(all_documents)} documents recuperes")
    return all_documents

In [ ]:
def transform_document(doc, ingestion_week: str):
    """
    Transforme un document AI Search en tuple pour le DataFrame.
    Synchronise avec le schema defini ci-dessus (38 colonnes).
    
    Args:
        doc: Document depuis AI Search
        ingestion_week: Semaine d'ingestion au format "2024-W49"
    """
    # Publication date
    publication_date = None
    if doc.get("publicationDate"):
        try:
            publication_date = datetime.fromisoformat(doc["publicationDate"].replace("Z", "+00:00"))
        except:
            pass
    elif doc.get("published"):
        try:
            publication_date = datetime.fromtimestamp(doc["published"] / 1000)
        except:
            pass
    
    # Crawled date
    crawled_at = None
    if doc.get("crawled"):
        try:
            crawled_at = datetime.fromtimestamp(doc["crawled"] / 1000)
        except:
            pass
    
    # Opening date
    opening_date = None
    if doc.get("openingDate"):
        try:
            opening_date = datetime.fromisoformat(doc["openingDate"].replace("Z", "+00:00"))
        except:
            pass
    
    now = datetime.now()
    
    return (
        # id
        doc.get("id"),
        # Article metadata
        doc.get("url", ""),
        doc.get("title", ""),
        doc.get("content", ""),
        publication_date,
        doc.get("language", ""),
        doc.get("origin", ""),
        doc.get("entities", ""),
        doc.get("topics", ""),
        # Vertical
        doc.get("vertical", ""),
        # Venue
        doc.get("venueName", ""),
        doc.get("city", ""),
        doc.get("country", ""),
        doc.get("zone", ""),
        doc.get("venueType", ""),
        doc.get("capacity"),
        # Project
        doc.get("projectType", ""),
        doc.get("projectPhase", ""),
        doc.get("openingYear"),
        opening_date,
        # Financial
        doc.get("investment"),
        doc.get("investmentCurrency", ""),
        # Stakeholders
        doc.get("investorOwnerManagement", ""),
        doc.get("architectConsultantContractor", ""),
        # Competitor intelligence
        doc.get("competitorNameMain", ""),
        doc.get("competitorNameOther", ""),
        doc.get("keyProductsInstalled", ""),
        doc.get("systemIntegrator", ""),
        doc.get("otherKeyPlayers", ""),
        # Additional
        doc.get("additionalInformation", ""),
        # Analysis
        doc.get("evaluationScore"),
        doc.get("auditOpportunity", False),
        doc.get("auditOpportunityReason", ""),
        # Metadata
        crawled_at,
        now,  # loaded_at
        now,  # updated_at
        ingestion_week,  # ingestion_week (ex: "2024-W49")
        # Deduplication (None par defaut, rempli par deduplicate_weekly.ipynb)
        None,  # is_duplicate
        None,  # is_suspected_duplicate
        None,  # duplicate_of
        None   # duplicate_score
    )

## 5. Extraction des données

In [ ]:
# Récupérer les documents selon le mode
documents = fetch_documents_from_ai_search(
    endpoint=AI_SEARCH_ENDPOINT,
    index=AI_SEARCH_INDEX,
    api_key=AI_SEARCH_KEY,
    mode=LOAD_MODE,
    hours_back=INCREMENTAL_HOURS
)

if not documents:
    print("⚠️ Aucun nouveau document à charger")
else:
    # Calculer la semaine d'ingestion (format ISO 8601: "2024-W49")
    ingestion_week = datetime.now().strftime("%Y-W%V")
    print(f"Semaine d'ingestion: {ingestion_week}")
    
    # Transformer les documents
    rows = [transform_document(doc, ingestion_week) for doc in documents]
    print(f"✅ {len(rows)} documents transformés")

## 6. Chargement selon le mode

In [ ]:
if documents:
    # Creer le DataFrame
    df_new = spark.createDataFrame(rows, schema)
    print(f"DataFrame cree: {df_new.count()} lignes")
    
    # =========================================================================
    # MODE: ONE-TIME LOAD
    # =========================================================================
    if LOAD_MODE == "one_time":
        print("\nONE-TIME LOAD: Creation initiale de la table...")
        
        df_new.write \
            .format("delta") \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .saveAsTable(TABLE_LANDING)
        
        print(f"Table '{TABLE_LANDING}' creee avec {df_new.count()} enregistrements")
    
    # =========================================================================
    # MODE: INCREMENTAL (DELTA)
    # =========================================================================
    elif LOAD_MODE == "incremental":
        print("\nINCREMENTAL: Merge des nouveaux documents...")
        
        # Verifier si la table existe
        table_exists = spark.catalog.tableExists(TABLE_LANDING)
        
        if not table_exists:
            print(f"   Table '{TABLE_LANDING}' n'existe pas, creation...")
            df_new.write \
                .format("delta") \
                .mode("overwrite") \
                .saveAsTable(TABLE_LANDING)
        else:
            # MERGE: Insert new, Update existing
            # Note: ingestion_week et colonnes deduplication ne sont PAS mises a jour
            #       - ingestion_week: conserve la semaine d'origine
            #       - is_duplicate, is_suspected_duplicate, duplicate_of, duplicate_score:
            #         geres par deduplicate_weekly.ipynb
            delta_table = DeltaTable.forName(spark, TABLE_LANDING)
            
            delta_table.alias("target").merge(
                df_new.alias("source"),
                "target.id = source.id"
            ).whenMatchedUpdate(
                set={
                    "article_url": "source.article_url",
                    "article_title": "source.article_title",
                    "article_content": "source.article_content",
                    "publication_date": "source.publication_date",
                    "language": "source.language",
                    "origin": "source.origin",
                    "entities": "source.entities",
                    "topics": "source.topics",
                    "vertical": "source.vertical",
                    "venue_name": "source.venue_name",
                    "city": "source.city",
                    "country": "source.country",
                    "zone": "source.zone",
                    "venue_type": "source.venue_type",
                    "capacity": "source.capacity",
                    "project_type": "source.project_type",
                    "project_phase": "source.project_phase",
                    "opening_year": "source.opening_year",
                    "opening_date": "source.opening_date",
                    "investment": "source.investment",
                    "investment_currency": "source.investment_currency",
                    "investor_owner_management": "source.investor_owner_management",
                    "architect_consultant_contractor": "source.architect_consultant_contractor",
                    "competitor_name_main": "source.competitor_name_main",
                    "competitor_name_other": "source.competitor_name_other",
                    "key_products_installed": "source.key_products_installed",
                    "system_integrator": "source.system_integrator",
                    "other_key_players": "source.other_key_players",
                    "additional_information": "source.additional_information",
                    "evaluation_score": "source.evaluation_score",
                    "audit_opportunity": "source.audit_opportunity",
                    "audit_opportunity_reason": "source.audit_opportunity_reason",
                    "crawled_at": "source.crawled_at",
                    "updated_at": "source.updated_at"
                    # Ne PAS mettre a jour:
                    # - loaded_at (date de premier chargement)
                    # - ingestion_week (semaine d'origine)
                    # - is_duplicate, is_suspected_duplicate, duplicate_of, duplicate_score
                    #   (geres par deduplicate_weekly.ipynb)
                }
            ).whenNotMatchedInsertAll().execute()
            
            print(f"Merge termine")
        
        # Stats
        total = spark.sql(f"SELECT COUNT(*) FROM {TABLE_LANDING}").collect()[0][0]
        print(f"   Total dans la table: {total} enregistrements")
    
    # =========================================================================
    # MODE: FULL REFRESH
    # =========================================================================
    elif LOAD_MODE == "full_refresh":
        print("\nFULL REFRESH: Rechargement complet...")
        
        # Ecraser la table curated (pas la landing)
        df_new.write \
            .format("delta") \
            .mode("overwrite") \
            .option("overwriteSchema", "true") \
            .saveAsTable(TABLE_CURATED)
        
        print(f"Table '{TABLE_CURATED}' recreee avec {df_new.count()} enregistrements")
        
        # Vacuum pour nettoyer les anciens fichiers
        spark.sql(f"VACUUM {TABLE_CURATED} RETAIN 168 HOURS")
        print("   Vacuum effectue")

## 7. Validation et Statistiques

In [ ]:
# Déterminer quelle table afficher
display_table = TABLE_CURATED if LOAD_MODE == "full_refresh" else TABLE_LANDING

print(f"📊 Statistiques de '{display_table}':\n")

# Counts
total = spark.sql(f"SELECT COUNT(*) FROM {display_table}").collect()[0][0]
opportunities = spark.sql(f"SELECT COUNT(*) FROM {display_table} WHERE audit_opportunity = true").collect()[0][0]
avg_score = spark.sql(f"SELECT AVG(evaluation_score) FROM {display_table} WHERE evaluation_score IS NOT NULL").collect()[0][0]

print(f"   📈 Total records: {total}")
print(f"   🎯 Opportunities (audit=true): {opportunities} ({opportunities/total*100:.1f}%)" if total > 0 else "   🎯 Opportunities: 0")
print(f"   📊 Average score: {avg_score:.1f}" if avg_score else "   📊 Average score: N/A")

In [ ]:
# Distribution par Vertical
print("\n📊 Distribution par Vertical:")
spark.sql(f"""
    SELECT 
        vertical,
        COUNT(*) as total,
        SUM(CASE WHEN audit_opportunity = true THEN 1 ELSE 0 END) as opportunities,
        ROUND(AVG(evaluation_score), 1) as avg_score
    FROM {display_table}
    WHERE vertical IS NOT NULL AND vertical != ''
    GROUP BY vertical
    ORDER BY total DESC
""").show(25, truncate=False)

In [ ]:
# Distribution par Zone
print("\n🌍 Distribution par Zone:")
spark.sql(f"""
    SELECT 
        zone,
        COUNT(*) as total,
        SUM(CASE WHEN audit_opportunity = true THEN 1 ELSE 0 END) as opportunities,
        ROUND(AVG(evaluation_score), 1) as avg_score
    FROM {display_table}
    WHERE zone IS NOT NULL AND zone != ''
    GROUP BY zone
    ORDER BY total DESC
""").show()

In [ ]:
# Distribution par Project Phase
print("\n🏗️ Distribution par Project Phase:")
spark.sql(f"""
    SELECT 
        project_phase,
        COUNT(*) as total,
        SUM(CASE WHEN audit_opportunity = true THEN 1 ELSE 0 END) as opportunities,
        ROUND(AVG(evaluation_score), 1) as avg_score
    FROM {display_table}
    WHERE project_phase IS NOT NULL AND project_phase != ''
    GROUP BY project_phase
    ORDER BY total DESC
""").show(15, truncate=False)

In [ ]:
# Top 10 Opportunités
print("\n🏆 Top 10 Opportunités (score le plus élevé):")
spark.sql(f"""
    SELECT 
        venue_name,
        city,
        country,
        vertical,
        project_phase,
        evaluation_score,
        opening_year
    FROM {display_table}
    WHERE audit_opportunity = true
    ORDER BY evaluation_score DESC
    LIMIT 10
""").show(truncate=30)

In [ ]:
# Aperçu des derniers chargements
print("\n🕐 Derniers enregistrements chargés:")
spark.sql(f"""
    SELECT 
        article_title,
        vertical,
        city,
        country,
        evaluation_score,
        audit_opportunity,
        loaded_at
    FROM {display_table}
    ORDER BY loaded_at DESC
    LIMIT 10
""").show(truncate=40)

## 8. Historique Delta (debug)

In [ ]:
# Historique des opérations Delta
if spark.catalog.tableExists(TABLE_LANDING):
    print(f"\n📜 Historique Delta de '{TABLE_LANDING}':")
    spark.sql(f"DESCRIBE HISTORY {TABLE_LANDING}").select(
        "version", "timestamp", "operation", "operationParameters"
    ).show(10, truncate=50)

---

## Pipeline complet

**Chaque semaine:**

1. **Azure Function Collector** `/api/feed` → Feedly → `q-raw-events`
2. **enrich_event** → Agent `lac-weak-signals` → AI Search (upsert) → `q-enriched-events`
3. **analyze_event** → Agent `lac-analyst-leads` → AI Search (merge) → `q-opportunities` (si opportunite)
4. **Ce Notebook Fabric** (`LOAD_MODE="one_time"` au deploiement, puis `"incremental"` en CRON hebdo)
   → AI Search → Lakehouse `landing_feedly_opportunities`
5. **deduplicate_weekly.ipynb** → Deduplication semantique avec Fabric AI Functions
6. **Power BI / Fabric** pour revue metier & priorisation

---

## Resume des modes

| Mode | Table cible | Action | Quand l'utiliser |
|------|-------------|--------|------------------|
| `one_time` | `landing_feedly_opportunities` | OVERWRITE | Deploiement initial |
| `incremental` | `landing_feedly_opportunities` | MERGE (upsert) | CRON hebdo |
| `full_refresh` | `curated_feedly_opportunities` | OVERWRITE | Ad hoc / mensuel |

---

## Colonnes de la table (38 colonnes)

| Categorie | Colonnes |
|-----------|----------|
| **ID** | `id` |
| **Article** | `article_url`, `article_title`, `article_content`, `publication_date`, `language`, `origin`, `entities`, `topics` |
| **Vertical** | `vertical` |
| **Venue** | `venue_name`, `city`, `country`, `zone`, `venue_type`, `capacity` |
| **Projet** | `project_type`, `project_phase`, `opening_year`, `opening_date` |
| **Financier** | `investment`, `investment_currency` |
| **Stakeholders** | `investor_owner_management`, `architect_consultant_contractor` |
| **Concurrents** | `competitor_name_main`, `competitor_name_other`, `key_products_installed`, `system_integrator`, `other_key_players` |
| **Additional** | `additional_information` |
| **Analyse** | `evaluation_score`, `audit_opportunity`, `audit_opportunity_reason` |
| **Metadata** | `crawled_at`, `loaded_at`, `updated_at`, `ingestion_week` |
| **Deduplication** | `is_duplicate`, `is_suspected_duplicate`, `duplicate_of`, `duplicate_score` |

---

## Deduplication

Les colonnes de deduplication sont **gerees par un notebook separe** (`deduplicate_weekly.ipynb`) qui utilise les Fabric AI Functions:

| Colonne | Type | Description |
|---------|------|-------------|
| `ingestion_week` | String | Semaine d'ingestion (ex: "2024-W49") |
| `is_duplicate` | Boolean | `true` si doublon confirme (score >= 0.90) |
| `is_suspected_duplicate` | Boolean | `true` si zone grise (0.85 <= score < 0.90) |
| `duplicate_of` | String | ID de l'article original (si doublon) |
| `duplicate_score` | Double | Score de similarite (0.0 a 1.0) |

**Texte compare pour embedding:** `title + venue_name + city + country`